In [1]:
import netCDF4
import numpy as np
import pandas as pd
import geopandas as gpd
import pycaret
from datetime import timezone, datetime, timedelta
from tqdm import tqdm
from shapely.geometry import Point

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
import os
import os
from pycaret.classification import *

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches

class SurfaceTypeUtils:
    surface_type_dict = {
        -1: "Ocean",
        0: "NaN",
        1: "Artifical",
        2: "Barely vegetated",
        3: "Inland water",
        4: "Crop",
        5: "Grass",
        6: "Shrub",
        7: "Forest"
    }
    ddm_antennas = {
        0: 'None',
        1: 'Zenith',
        2: 'LHCP',
        3: 'RHCP',
    }


KeyboardInterrupt: 

In [ ]:

class GeoUtils:
    def __init__(self, world_shapefile_path):
        self.world = gpd.read_file(world_shapefile_path)

    @staticmethod
    def add_seconds(time, seconds):
        timestamp = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        new_timestamp = timestamp + timedelta(seconds=seconds)
        return new_timestamp.strftime("%Y-%m-%d %H:%M:%S")

    def is_land(self, lat, lon):
        point = Point(lon, lat)
        return any(self.world.contains(point))

    @staticmethod
    def check_ocean_and_land(lst):
        has_ocean = -1 in lst
        has_land = any(1 <= num <= 7 for num in lst)
        return has_ocean and has_land

    @staticmethod
    def fill_and_filter(arr):
        mask_all_nan = np.all(np.isnan(arr), axis=(2, 3))
        arr_filled = arr.copy()
        for i in range(arr.shape[0]):
            nan_indices = np.where(mask_all_nan[i])[0]
            if len(nan_indices) > 0:
                valid_indices = np.where(~mask_all_nan[i])[0]
                if len(valid_indices) > 0:
                    mean_matrix = np.nanmean(arr[i, valid_indices, :, :], axis=0)
                    arr_filled[i, nan_indices, :, :] = mean_matrix
        mask_discard = np.all(mask_all_nan, axis=1)
        arr_filtered = arr_filled[~mask_discard]
        return arr_filtered, list(np.where(mask_discard.astype(int) == 1)[0])


In [ ]:

class NetCDFPreprocessor:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.netcdf_file_list = os.listdir(root_dir)

    def preprocess(self, f):
        raw_counts = np.array(f.variables['raw_counts'])
        ac_alt_2d = np.repeat(np.array(f.variables['ac_alt'])[:, np.newaxis], 20, axis=1)
        distance_2d = (ac_alt_2d - f.variables['sp_alt'][:]) / np.cos(np.deg2rad(f.variables['sp_inc_angle'][:]))
        copol = f.variables['sp_rx_gain_copol'][:]
        xpol = f.variables['sp_rx_gain_xpol'][:]
        snr = f.variables['ddm_snr'][:]
        dist = distance_2d[:]
        keep_mask = (copol >= 5) & (xpol >= 5) & (snr > 0) & ((dist >= 2000) & (dist <= 10000)) & (~np.isnan(copol.data) & ~np.isnan(xpol.data) & ~np.isnan(snr.data) & ~np.isnan(dist.data))
        to_keep_indices = np.argwhere(keep_mask)
        filtered_raw_counts = [raw_counts[i, j] for i, j in to_keep_indices]
        output_array = np.full(raw_counts.shape, np.nan, dtype=np.float32)

        for idx, (i, j) in enumerate(to_keep_indices):
            output_array[i, j] = filtered_raw_counts[idx]
        raw_counts_filtered = output_array.copy()

        del output_array

        ddm_data_dict = {
            'Raw_Counts': raw_counts_filtered.reshape(raw_counts_filtered.shape[0]*raw_counts_filtered.shape[1], raw_counts_filtered.shape[2], raw_counts_filtered.shape[3]),
        }
        keep_indices = np.where(
            np.all(~np.isnan(ddm_data_dict['Raw_Counts']), axis=(1, 2)) & (np.sum(ddm_data_dict['Raw_Counts'], axis=(1, 2)) > 0)
        )[0]
        fit_data = np.array([ddm_data_dict['Raw_Counts'][f].ravel() for f in keep_indices])
        surface_types = f.variables["sp_surface_type"][:]
        surface_types = np.nan_to_num(surface_types, nan=0)
        surface_types_unravelled = surface_types.ravel()
        label_data = [1 if surface_type in np.arange(1, 8) else 0 for surface_type in surface_types_unravelled]
        label_data = [label_data[l] for l in range(len(label_data)) if l in keep_indices]
        return fit_data, label_data


    def process_all_files(self, save = bool):
        
        from sklearn.model_selection import train_test_split
        full_data = []
        full_labels = []
        counter = 0
        for file_name in tqdm(self.netcdf_file_list, desc="Processing files"):
            if not file_name.endswith('.nc'):
                continue
            f = netCDF4.Dataset(f'{self.root_dir}{file_name}')
            data, labels = self.preprocess(f)
            full_data.append(data)
            full_labels.append(labels)
            counter += 1
            if counter == 70:
                break  # Limita il numero di file processati per test
                
        # Trova gli indici degli elementi di full_data con seconda dimensione uguale a 200
        valid_indices = [i for i, arr in enumerate(full_data) if arr.ndim == 2 if arr.shape[1] == 200]

        # Applica la selezione a full_data e full_labels
        full_data_clean = [full_data[i] for i in valid_indices]
        full_labels_clean = [full_labels[i] for i in valid_indices]

        #full_data_clean = np.vstack(full_data_clean) Unable to allocate memory for this operation
        #full_labels_clean = np.hstack(full_labels_clean) Unable to allocate memory for this operation
        #print(f"Shape of full_data_clean before chunking and sampling: {np.array(full_data_clean).shape}")
        #print(f"Shape of full_labels_clean before chunking and sampling: {np.array(full_labels_clean).shape}")
        
        # Chunking 
        
        import pyarrow as pa
        import pyarrow.parquet as pq
        from sklearn.model_selection import train_test_split
        os.makedirs('processed_data/binary_classification', exist_ok=True)

        chunk_size = 250 # dimensione del chunk in numero di campioni
        sample_fraction = 0.05  # frazione di dati da campionare per ogni chunk

        full_data_sampled = []
        full_labels_sampled = []

        num_chunks = int(np.ceil(len(full_data_clean) / chunk_size))
        for idx in range(num_chunks):
            start = idx * chunk_size
            end = min((idx + 1) * chunk_size, len(full_data_clean))
            chunk_data = np.vstack(full_data_clean[start:end])
            chunk_labels = np.vstack(full_labels_clean[start:end])
            
            print(f"Chunk {idx + 1}/{num_chunks} processed with shape {chunk_data.shape} and labels shape {chunk_labels.shape}")

            # Salva ogni chunk come file parquet separato
            fit_data_df = pd.DataFrame(chunk_data)
            labels_df = pd.DataFrame(chunk_labels, columns=['label'])

            table_fit = pa.Table.from_pandas(fit_data_df, preserve_index=False)
            table_labels = pa.Table.from_pandas(labels_df, preserve_index=False)

            pq.write_table(
                table_fit,
                f'processed_data/binary_classification/fit_data_chunk_{idx}.parquet',
                compression='zstd',
                use_dictionary=True,
            )
            pq.write_table(
                table_labels,
                f'processed_data/binary_classification/labels_chunk_{idx}.parquet',
                compression='zstd',
                use_dictionary=True,
            )

        # Imposta la frazione di dati da campionare per ogni chunk (es: 0.2 per il 20%)
        
            _, X_sampled, _, y_sampled = train_test_split(
                chunk_data, chunk_labels, 
                test_size=sample_fraction, 
                stratify=chunk_labels, 
                random_state=42
            )
            del chunk_data, chunk_labels, table_fit, table_labels  # Libera memoria
            
            full_data_sampled.append(X_sampled)
            full_labels_sampled.append(y_sampled)

        full_data_sampled_stratified = np.vstack(full_data_sampled)
        full_labels_sampled_stratified = np.hstack(full_labels_sampled)
        print(f"Shape of sampled data after chunking and sampling: {np.array(full_data_sampled_stratified).shape}")
        print(f"Shape of sampled labels after chunking and sampling: {np.array(full_labels_sampled_stratified).shape}")
        
        # Crea la cartella processed_data se non esiste
        os.makedirs('processed_data/binary_classification', exist_ok=True)

        # Salva fit_data in formato parquet ottimizzato
        fit_data_df = pd.DataFrame(full_data_sampled_stratified)
        table_fit = pa.Table.from_pandas(fit_data_df, preserve_index=False)
        pq.write_table(
            table_fit,
            'processed_data/binary_classification/fit_data_stratified_binary.parquet',
            compression='zstd',
            use_dictionary=True,
            
        )

        # Salva labels in formato parquet ottimizzato
        labels_df = pd.DataFrame(full_labels_sampled_stratified, columns=['label'])
        table_labels = pa.Table.from_pandas(labels_df, preserve_index=False)
        pq.write_table(
            table_labels,
            'processed_data/binary_classification/labels_stratified_binary.parquet',
            compression='zstd',
            use_dictionary=True,
            
        )

        return full_data_sampled_stratified, full_labels_sampled_stratified

class ModelTrainer:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.final_model = None

    def train(self, model_search=True):
        os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"
        if model_search:
            scaler = MinMaxScaler()
            fit_data_scaled = scaler.fit_transform(self.data)
            clf = setup(data=fit_data_scaled,
                        target=self.labels,
                        pca=True,
                        pca_method='incremental',
                        use_gpu=True
                        )
            best_models = compare_models(n_select=5)
            best_model = best_models[0]
            print(f"Il modello migliore è: {best_model}")
            tuned_model = tune_model(best_model,
                                    optimize='Accuracy',
                                    n_iter=10,
                                    search_library='optuna',
                                    search_algorithm='tpe',
                                    choose_better=True)
            print("Valutazione del modello ottimizzato:")
            evaluate_model(tuned_model)
            best_params = tuned_model.get_params()
            print("Migliori iperparametri trovati:")
            for param, value in best_params.items():
                print(f"{param}: {value}")
            self.final_model = finalize_model(tuned_model)
            save_model(self.final_model, 'best_classification_model')
            # loaded_model = load_model('best_classification_model')

In [ ]:
ROOT_DIR = 'D:/GREAT/machine_learning/data/Rongowai/'


read_from_backup = False
if read_from_backup:
    import polars as pl

    # Leggi i file parquet con polars
    fit_data_pl = pl.read_parquet('processed_data/fit_data_binary.parquet')
    labels_pl = pl.read_parquet('processed_data/labels_binary.parquet')

    # Trasforma in numpy array
    fit_data = fit_data_pl.to_numpy()
    labels = labels_pl['label'].to_numpy()
else:
    preprocessor = NetCDFPreprocessor(root_dir=ROOT_DIR)
    fit_data, labels = preprocessor.process_all_files(save=True)

In [ ]:
len(fit_data), len(labels)

In [ ]:
from sklearn.model_selection import train_test_split

# Esegui il campionamento stratificato, prendi 1-test_size%  del campione
fit_data_sample, _, labels_sample, _ = train_test_split(
    fit_data, labels, test_size=0.95, stratify=labels, random_state=42
)

In [ ]:
len(fit_data_sample), len(labels_sample)

In [ ]:

model_trainer = ModelTrainer(data=fit_data, labels=labels)
model_trainer.train(model_search=True)